In [1]:
using SymPy
using LinearAlgebra

δ = sympy.eye(2); 
ϵ = sympy.Matrix([0 1; -1 0]);
g = sympy.diag(-1,1,1,1);

## 泡利矩阵

In [2]:
σ_0 = [1 0;0 1]/sqrt(Sym(2));
σ_1 = [0 1;1 0]/sqrt(Sym(2));
σ_2 = [0 -IM;IM 0]/sqrt(Sym(2));
σ_3 = [1 0;0 -1]/sqrt(Sym(2));
σ=[σ_0,σ_1,σ_2,σ_3];
σ=[σ[μ][Σ,Σ1] for μ in 1:4,Σ in 1:2 ,Σ1 in 1:2];

[σ[μ,Σ,Σ1] for Σ in 1:2, Σ1 in 1:2, μ in 1:4]

2×2×4 Array{Sym,3}:
[:, :, 1] =
 sqrt(2)/2          0
         0  sqrt(2)/2

[:, :, 2] =
         0  sqrt(2)/2
 sqrt(2)/2          0

[:, :, 3] =
           0  -sqrt(2)*I/2
 sqrt(2)*I/2             0

[:, :, 4] =
 sqrt(2)/2           0
         0  -sqrt(2)/2

## 零模四维矢量$k^a$

In [3]:
@vars r positive=true θ ϕ  x real=true

k = [r, r*sin(θ)*cos(ϕ), r*sin(θ)*sin(ϕ), r*cos(θ)]

4-element Array{Sym,1}:
               r
 r*sin(θ)*cos(ϕ)
 r*sin(θ)*sin(ϕ)
        r*cos(θ)

In [4]:
# 验证零模

sum(g[μ,υ]*k[μ]*k[υ] for μ in 1:4, υ in 1:4) .|> simplify  

0

##  旋量$\xi^A$的分量


确定依据【精确到相差一个旗因子】：

$$k^\mu\sigma_\mu^{\ \ AA'}=\xi^A\bar{\xi}^{A'}$$

In [5]:
@vars a1 a2 positive=true α α1 α2 real=true

# 待定

ξ =[a1*exp(IM*α1),a2*exp(IM*α2)]

2-element Array{Sym,1}:
 a1*exp(I*α1)
 a2*exp(I*α2)

In [6]:
#  待解方程组

eqs = [sum(k[μ]*σ[μ,Σ,Σ1] for μ in 1:4)-ξ[Σ]*conj(ξ[Σ1]) for Σ in 1:2, Σ1 in 1:2];
eqs = [eqs[i,j] for i in 1:2, j in 1:2 if i <= j];
eqs

3-element Array{Sym,1}:
                                              -a1^2 + sqrt(2)*r*cos(θ)/2 + sqrt(2)*r/2
 -a1*a2*exp(I*α1)*exp(-I*α2) - sqrt(2)*I*r*sin(θ)*sin(ϕ)/2 + sqrt(2)*r*sin(θ)*cos(ϕ)/2
                                              -a2^2 - sqrt(2)*r*cos(θ)/2 + sqrt(2)*r/2

In [7]:
# 方程组求解

rs = solve(eqs,[a1,a2,α1])[4] .|>  
    (expr->expr(θ=>2*θ))  .|>  simplify.|>   # 三角化简：临时变量替换
    (expr->refine(expr, Q.positive(cos(θ)) & Q.positive(sin(θ))))  .|>  simplify .|>  # 去除绝对值
    (expr->expr(IM*(α2-ϕ)=>x)) .|>  simplify .|>  #  对数指数化简：临时变量替换
    (expr->expr(x=>IM*(α2-ϕ),θ=>θ//2)); # 变量复原
rs = [rs[i] for i in 1:3]

3-element Array{Sym,1}:
 2^(1/4)*sqrt(r)*cos(θ/2)
 2^(1/4)*sqrt(r)*sin(θ/2)
                   α2 - ϕ

In [8]:
#  旋量分量确定 

ξ=ξ .|> (expr->expr(a1=>rs[1],a2=>rs[2],α1=>rs[3]))   .|>  simplify .|>
        (expr->expr(α2=>ϕ//2+α//2))  # 不确定因子调整

2-element Array{Sym,1}:
 2^(1/4)*sqrt(r)*exp(I*(α/2 - ϕ/2))*cos(θ/2)
 2^(1/4)*sqrt(r)*exp(I*(α/2 + ϕ/2))*sin(θ/2)

In [9]:
# 验证

rs = [sum(k[μ]*σ[μ,Σ,Σ1] for μ in 1:4)-ξ[Σ]*conj(ξ[Σ1]) for Σ in 1:2, Σ1 in 1:2] 

rs  .|> real .|> simplify

2×2 Array{Sym,2}:
 0  0
 0  0

In [10]:
rs  .|> imag  .|> simplify

2×2 Array{Sym,2}:
 0  0
 0  0

##  旋量$\eta^A$的分量

确定依据【精确到可加上$c \xi^A$】:

$$\xi_A\eta^A=1$$

In [11]:
@vars η1 η2 c

#  η1 可以被 c*ξ吸收掉
#  η2  待定
#   c  手工调整，目标：使得两个分量的简单度一致
η =[0,η2] + c*ξ

2-element Array{Sym,1}:
      2^(1/4)*c*sqrt(r)*exp(I*(α/2 - ϕ/2))*cos(θ/2)
 2^(1/4)*c*sqrt(r)*exp(I*(α/2 + ϕ/2))*sin(θ/2) + η2

In [12]:
eqs = sum(ϵ[Σ,Ω]*ξ[Σ]*η[Ω] for Σ in 1:2, Ω in 1:2)-1

            ⎛α   ϕ⎞    ⎛α   ϕ⎞                          ⎛              ⎛α   ϕ⎞
          ⅈ⋅⎜─ - ─⎟  ⅈ⋅⎜─ + ─⎟                          ⎜            ⅈ⋅⎜─ + ─⎟
            ⎝2   2⎠    ⎝2   2⎠    ⎛θ⎞    ⎛θ⎞   4 ___    ⎜4 ___         ⎝2   2⎠
- √2⋅c⋅r⋅ℯ         ⋅ℯ         ⋅sin⎜─⎟⋅cos⎜─⎟ + ╲╱ 2 ⋅√r⋅⎜╲╱ 2 ⋅c⋅√r⋅ℯ         
                                  ⎝2⎠    ⎝2⎠            ⎝                     

            ⎞    ⎛α   ϕ⎞           
            ⎟  ⅈ⋅⎜─ - ─⎟           
    ⎛θ⎞     ⎟    ⎝2   2⎠    ⎛θ⎞    
⋅sin⎜─⎟ + η2⎟⋅ℯ         ⋅cos⎜─⎟ - 1
    ⎝2⎠     ⎠               ⎝2⎠    

In [13]:
rs = solve(eqs,η2)[1]

      ⅈ⋅(-α + ϕ)
      ──────────
 3/4      2     
2   ⋅ℯ          
────────────────
          ⎛θ⎞   
  2⋅√r⋅cos⎜─⎟   
          ⎝2⎠   

In [14]:
# 通过手工逐步调整c ，使得最复杂的分量变得更简单，直到两个分量的简单度一致 

η=η .|> (expr->expr(η2=>rs)) .|> simplify .|>
    (expr->expr(θ=>2*θ)) .|>  simplify  .|>   # 临时变量替换
    (expr->expr(c=>c*tan(θ)/r)) .|>  simplify  .|>  # 目标：逐步将第二个分量简化
    (expr->expr(c=>c*exp(-IM*α))) .|>  simplify .|>   
    (expr->expr(c=>-sqrt(Sym(2))/2)) .|>  simplify  .|>
    (expr->expr(θ=>θ//2))   .|> simplify  # 临时变量复原

2-element Array{Sym,1}:
 -2^(3/4)*exp(-I*(α + ϕ)/2)*sin(θ/2)/(2*sqrt(r))
  2^(3/4)*exp(-I*(α - ϕ)/2)*cos(θ/2)/(2*sqrt(r))

In [15]:
# 验证

sum(ϵ[Σ,Ω]*ξ[Σ]*η[Ω] for Σ in 1:2, Ω in 1:2)  .|>sympy.expand_trig .|> simplify 

1

## 零模四维矢量$l^a$

确定依据：

$$l^\mu\sigma_\mu^{\ \ AA'}= l^{AA'}=\xi^A\bar{\eta}^{A'}+\eta^A\bar{\xi}^{A'}$$

In [16]:
@vars l0 l1 l2 l3 real=true

# 待定
l=[l0,l1,l2,l3]

4-element Array{Sym,1}:
 l0
 l1
 l2
 l3

In [17]:
eqs = [ξ[Σ]*conj(η[Σ1])+η[Σ]*conj(ξ[Σ1])-sum(l[μ]*σ[μ,Σ,Σ1] for μ in 1:4) 
            for Σ in 1:2, Σ1 in 1:2] .|> simplify 
eqs = [eqs[i,j] for i in 1:2, j in 1:2 if i<=j];
eqs_r = eqs .|> real .|> simplify ;
eqs_i = eqs .|> imag .|> simplify ;
eqs_i = [eqs_i[i] for i in 1:3 if eqs_i[i]!=0]
eqs = vcat(eqs_r,eqs_i)

4-element Array{Sym,1}:
            -sqrt(2)*l0/2 - sqrt(2)*l3/2 - sin(θ)*cos(α)
 -sqrt(2)*l1/2 - 2*sin(θ/2)^2*cos(α)*cos(ϕ) + cos(α - ϕ)
            -sqrt(2)*l0/2 + sqrt(2)*l3/2 + sin(θ)*cos(α)
  sqrt(2)*l2/2 + 2*sin(θ/2)^2*sin(ϕ)*cos(α) + sin(α - ϕ)

In [18]:
rs = solve(eqs,l)

Dict{Any,Any} with 4 entries:
  l0 => 0
  l2 => -sqrt(2)*(2*sin(θ/2)^2*sin(ϕ)*cos(α) + sin(α - ϕ))
  l3 => -sqrt(2)*sin(θ)*cos(α)
  l1 => sqrt(2)*(-2*sin(θ/2)^2*cos(α)*cos(ϕ) + cos(α - ϕ))

In [19]:
l = l .|> (expr->expr(l0=>rs[l0],l1=>rs[l1],l2=>rs[l2],l3=>rs[l3]))   .|>  
       (expr->expr((sin(θ//2))^2=>(1-cos(θ))//2)) .|>sympy.expand_trig .|> simplify 

4-element Array{Sym,1}:
                                               0
  sqrt(2)*(sin(α)*sin(ϕ) + cos(α)*cos(θ)*cos(ϕ))
 sqrt(2)*(-sin(α)*cos(ϕ) + sin(ϕ)*cos(α)*cos(θ))
                          -sqrt(2)*sin(θ)*cos(α)

In [20]:
sum(g[μ,υ]*l[μ]*l[υ] for μ in 1:4, υ in 1:4) .|> simplify 

2

## 旗$F^{ab}$

$$F^{ab}=k^a l^b- l^a k^b$$

In [21]:
F = [k[μ]*l[υ]-l[μ]*k[υ] for μ in 1:4, υ in 1:4] .|>sympy.expand_trig .|> simplify 

4×4 Array{Sym,2}:
                                                 0  …                           -sqrt(2)*r*sin(θ)*cos(α)
 -sqrt(2)*r*(sin(α)*sin(ϕ) + cos(α)*cos(θ)*cos(ϕ))     -sqrt(2)*r*(sin(α)*sin(ϕ)*cos(θ) + cos(α)*cos(ϕ))
  sqrt(2)*r*(sin(α)*cos(ϕ) - sin(ϕ)*cos(α)*cos(θ))      sqrt(2)*r*(sin(α)*cos(θ)*cos(ϕ) - sin(ϕ)*cos(α))
                           sqrt(2)*r*sin(θ)*cos(α)                                                     0

In [22]:
# 验证

det(F)  .|> sympy.expand_trig .|> simplify 

0